# Read files summarising field work and update database
These Excel workbooks were imported on February 2022.

The scripts documented here have been created to:

- Read data from spreadsheets with field-work data
- Create records for data import into the database
- Insert or update records in the database


## Set-up
Load libraries 

In [2]:
import openpyxl
from pathlib import Path
import os
from datetime import datetime
from configparser import ConfigParser
import psycopg2
from psycopg2.extras import DictCursor
from psycopg2.extensions import AsIs
import re
#import postgis
import pandas as pd
import copy

Define path to workbooks

In [3]:
repodir = Path("../../") 
inputdir = repodir / "data" / "field-form"

 ### DB connection parameters and helper functions
 
 Database credentials are stored in a database.ini file

In [4]:
filename = repodir / 'secrets' / 'database.ini'
section = 'aws-lght-sl'

# create a parser
parser = ConfigParser()
# read config file
parser.read(filename)

# get section, default to postgresql
db = {}
if parser.has_section(section):
    params = parser.items(section)
    for param in params:
        db[param[0]] = param[1]
else:
    raise Exception('Section {0} not found in the {1} file'.format(section, filename))

params = db

Define a function to batch process insert or update queries:

In [5]:
def batch_upsert(params,table,records,keycol,idx, execute=False,useconn=None):
    if useconn is None:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params)
    else:
        conn = useconn
    cur = conn.cursor()
    #postgis.register(cur)
    updated_rows=0
    errors=dict()
    for record in records:
        if len(record.keys())>len(keycol):
            if 'geom' in record.keys():
                the_geom=record['geom']
                record['geom']='GEOMSTR'
            if idx is not None:
                qrystr = "INSERT INTO %s (%s) values %s ON CONFLICT ON CONSTRAINT %s DO UPDATE SET %s"
                upd=list()
                for k in record.keys():
                    if k not in keycol:
                        upd.append("{col}=EXCLUDED.{col}".format(col=k))
                qry = cur.mogrify(qrystr, (AsIs(table),
                                AsIs(','.join(record.keys())),
                                tuple(record.values()),
                                AsIs(idx),
                                AsIs(','.join(upd))
                               ))
            else:
                qrystr = "INSERT INTO %s (%s) values %s ON CONFLICT DO NOTHING"
                qry = cur.mogrify(qrystr, (AsIs(table),
                                AsIs(','.join(record.keys())),
                                tuple(record.values())
                               ))

            if 'geom' in record.keys():
                qry=qry.decode('utf-8')
                qry=qry.replace("'GEOMSTR'",the_geom)
                record['geom']=the_geom

            if execute:
               # try:
                cur.execute(qry)
               # except psycopg2.errors.CheckViolation as error:
               #     if type(error) in errors.keys():
               #         errors[type(error)]=errors[type(error)]+1
                #    else:
                #        errors[type(error)]=1
                #except psycopg2.errors.CheckViolation as error:
                #    print(error)
                if cur.rowcount > 0:
                    updated_rows = updated_rows + cur.rowcount
            else:
                print(qry)
            
    conn.commit()        
    cur.close()
    print("%s rows updated" % (updated_rows))
    print(errors)
    if useconn is None and conn is not None:
        conn.close()
        print('Database connection closed.')

        

Just a test with random data, use `execute=False` to print the query:

In [6]:
record={'site_label':'test','geom':"ST_GeomFromText('POINT(1 2)', 4326)"}
batch_upsert(params,"form.field_site",(record,),keycol=('site_label',), idx='field_site_pkey1',execute=False)

Connecting to the PostgreSQL database...
INSERT INTO form.field_site (site_label,geom) values ('test', ST_GeomFromText('POINT(1 2)', 4326)) ON CONFLICT ON CONSTRAINT field_site_pkey1 DO UPDATE SET geom=EXCLUDED.geom
0 rows updated
{}
Database connection closed.


In [7]:
batch_upsert(params,"form.field_site",(record,),keycol=('site_label',), idx=None,execute=False)

Connecting to the PostgreSQL database...
INSERT INTO form.field_site (site_label,geom) values ('test', ST_GeomFromText('POINT(1 2)', 4326)) ON CONFLICT DO NOTHING
0 rows updated
{}
Database connection closed.


### List of valid sites and visits

In [8]:
conn = psycopg2.connect(**params)
cur = conn.cursor(cursor_factory=DictCursor)
cur.execute("select distinct visit_id,visit_date FROM form.field_visit")
all_visits=cur.fetchall()
cur.close()
conn.close()

In [9]:
all_visits[1]['visit_date']

datetime.date(2020, 1, 15)

### Read valid vegetation classes from spreadsheet

## Read workbooks
Each spreadsheet has a slightly different structure, so these scripts have to be adapted for each case.

### List of workbooks/spreadsheets in directory

In [10]:
avail_files = os.listdir(inputdir)
#avail_files

In [11]:
valid_files = ['SthnNSWRF_data_bionet2.xlsx',
               'UNSWFireVegResponse_UplandBasalt_AlexThomsen+DK.xlsx',
               'UNSW_VegFireResponse_RMK_reformat_Sep2021a.xlsx',
               'UNSW_VegFireResponse_DataEntry_Yatteyattah all +DK +Milton_revisedfields_Mar2022.xlsx',
               'UNSW_VegFireResponse_KNP AlpAsh_firehistupdate.xlsx',
               'UNSW_VegFireResponse_AlpineBogs_reformat_Sep2021.xlsx',
               'RobertsonRF_data_bionet2.xlsx',
               'Fire response quadrat survey Newnes Nov2020_DK_revised IDs+AllNovData.xlsm']

In [12]:
for ff in valid_files:
    print(ff in avail_files)

True
True
True
True
True
True
True
True


Here we create an index of worksheets and column headers for each file

In [13]:
wbindex=dict()
for workbook_name in valid_files:
    inputfile=inputdir / workbook_name
    # using data_only=True to get the calculated cell values
    wb = openpyxl.load_workbook(inputfile,data_only=True)
    wbindex[workbook_name]=dict()
    for ws in wb.worksheets:
        wbindex[workbook_name][ws._WorkbookChild__title]=[list(),list()]
        for k in range(1,ws.max_column):
            wbindex[workbook_name][ws._WorkbookChild__title][0].append(ws.cell(row=1,column=k).value)
            wbindex[workbook_name][ws._WorkbookChild__title][1].append(ws.cell(row=2,column=k).value)
        

In [14]:
wbindex.keys()

dict_keys(['SthnNSWRF_data_bionet2.xlsx', 'UNSWFireVegResponse_UplandBasalt_AlexThomsen+DK.xlsx', 'UNSW_VegFireResponse_RMK_reformat_Sep2021a.xlsx', 'UNSW_VegFireResponse_DataEntry_Yatteyattah all +DK +Milton_revisedfields_Mar2022.xlsx', 'UNSW_VegFireResponse_KNP AlpAsh_firehistupdate.xlsx', 'UNSW_VegFireResponse_AlpineBogs_reformat_Sep2021.xlsx', 'RobertsonRF_data_bionet2.xlsx', 'Fire response quadrat survey Newnes Nov2020_DK_revised IDs+AllNovData.xlsm'])

In [15]:
wbindex['SthnNSWRF_data_bionet2.xlsx'].keys()

dict_keys(['Site', 'Fire', 'Structure', 'Floristics', 'Reference', 'Info', 'Sheet1'])

### Fire intensity

We want to add the informaiton on the fire intensity. This is recorded in the worksheet 'Fire' next to the fire history. We have to add a column to the worksheet for the date of the sampling (as the last column on the right), otherwise we won't be able to match this information to the sampling visit.

We will use the variables:
- 'scorch height' in m, 
- 'tree foliage biomass consumed' in %,
- 'shrub foliage biomass consumed' in %,
- 'ground foliage biomass consumed' in %,
- 'tree foliage scorch' in %,
- 'shrub foliage scorch' in %,
- 'herb foliage scorch' in %




#### Function definitions

In [80]:
def read_fire_intensity(filepath,workbook,worksheet,col_definitions):
    wb = openpyxl.load_workbook(filepath / workbook,data_only=True)
    ws = wb[worksheet]
    triplet=('best','lower','upper')
    records=list()
    for row in range(2,ws.max_row+1):
        visitid=ws.cell(row,col_definitions['visit_id']).value
        if visitid is not None and visitid != 'Site':
            visitdate=ws.cell(row,col_definitions['visit_date']).value
            if isinstance(visitdate,datetime):
                visitdate=visitdate.date()
            elif visitdate is None:
                visitdate='NULL'
            else:
                visitdate=datetime.strptime(visitdate, '%d/%m/%Y').date()
            
            record={'visit_id': visitid,
                    'visit_date': visitdate,
                   'comment':list()}
            #print(record)
            for var in ('scorch height',
                        'tree foliage biomass consumed', 'shrub foliage biomass consumed', 'ground foliage biomass consumed',
                        'tree foliage scorch', 'shrub foliage scorch', 'herb foliage scorch',
                        'peat extent burnt','peat depth burnt'
                       ):
                if var in col_definitions.keys():
                    record1=copy.deepcopy(record)
                    record1['measured_var']=var
                    if var == 'scorch height':
                        record1['units']='m'
                    elif var == 'peat depth burnt':
                        record1['units']='cm'
                    else:
                        record1['units']='%'
                    for k in range(len(col_definitions[var])):
                        val=ws.cell(row,col_definitions[var][k]).value
                        if val is not None and val != 'NA':
                            if triplet[k]=='lower' and 'best' in record1.keys() and val > record1['best']:
                                record1['lower']=record1['best']
                                record1['comment'].append('lower bound given as %s but greater than best estimate' % val)
                            if triplet[k]=='upper' and 'best' in record1.keys() and val < record1['best']:
                                record1['comment'].append('upper bound given as %s but less than best estimate' % val)
                                record1['upper']=record1['best']
                            else:
                                record1[triplet[k]]=val
                    #print(record1)
                    if 'comment' in record1.keys() and len(record1['comment'])==0:
                        record1.pop('comment')
                    records.append(record1)
    return records


Add raw measurements for a single variable

In [58]:
def read_twig_diameters(filepath,workbook,worksheet,col_definitions):
    wb = openpyxl.load_workbook(filepath / workbook,data_only=True)
    ws = wb[worksheet]
    records=list()
    for row in range(2,ws.max_row+1):
        visitid=ws.cell(row,col_definitions['visit_id']).value
        if visitid is not None and visitid != 'Site':
            visitdate=ws.cell(row,col_definitions['visit_date']).value
            if isinstance(visitdate,datetime):
                visitdate=visitdate.date()
            elif visitdate is None:
                visitdate='NULL'
            else:
                visitdate=datetime.strptime(visitdate, '%d/%m/%Y').date()
             
            for var in ('twig diameter',):
                if var in col_definitions.keys():
                    record={'visit_id': visitid,
                        'visit_date': visitdate,
                        'measured_variable': var,
                        'units':'mm'
                       }
                    for k in range(len(col_definitions[var])):
                        record1=copy.deepcopy(record)
                        val=ws.cell(row,col_definitions[var][k]).value
                        if val is not None and val != 'NA':
                            record1['single_value']=val
                            records.append(record1)
    return records


#### Southern NSW Rainforest
Manual updates: add a visit_date column at the end of the table

In [18]:
filename='SthnNSWRF_data_bionet2.xlsx'
worksheet='Fire'
wbindex[filename][worksheet][0]

['Site',
 'Replicate',
 'Date of last fire dd/mm/yyyy',
 'Date of penultimate fire',
 'Date of earlier fire',
 'How date inferred1',
 'How date inferred2',
 'How date inferred3',
 'Ignition cause1',
 'Ignition cause2',
 'Ignition cause3',
 'Scorch hgt (m) min',
 'Scorch hgt (m) mas',
 'Scorch hgt (m) mode',
 '% Tree foliage scorch',
 "% Tree foliage c'sume",
 '% Shb foliage scorch',
 "% Shb foliage c'sume",
 '% Herb layer foliage scorch',
 "% Herb layer foliage c'sume",
 'Twig diam (mm) 1',
 'Twig diam (mm) 2',
 'Twig diam (mm) 3',
 'Twig diam (mm) 4',
 'Twig diam (mm) 5',
 'Twig diam (mm) 6',
 'Twig diam (mm) 7',
 'Twig diam (mm) 8',
 'Twig diam (mm) 9',
 'Twig diam (mm) 10',
 'Peat depth burnt (cm)',
 'Peat extent burnt %quad',
 'Peat extent unburnt %quad']

In [19]:
len(wbindex[filename][worksheet][0])

33

In [20]:
col_def={'visit_id':1, 'visit_date':34, 'scorch height':(14,12,13),
         'tree foliage biomass consumed':(16,), 
         'shrub foliage biomass consumed':(18,), 
         'ground foliage biomass consumed':(20,),
        'tree foliage scorch':(15,), 
         'shrub foliage scorch':(17,), 
         'herb foliage scorch':(19,)}

In [21]:
records=read_fire_intensity(inputdir,filename,worksheet,col_def)

In [22]:
records[1]

{'visit_id': 'UppClyde1',
 'visit_date': datetime.date(2021, 11, 29),
 'measured_var': 'tree foliage biomass consumed',
 'units': '%',
 'best': 0}

In [23]:
batch_upsert(params,"form.field_visit_vegetation_estimates",records,
             keycol=('visit_id','visit_date','measured_var'), 
             idx='field_visit_vegetation_estimates_pkey',execute=True)


Connecting to the PostgreSQL database...
45 rows updated
{}
Database connection closed.


#### Upland Basalt
Added a column for `Visit date` from the `Site` worksheet, changed date format to match the day/month/year format

In [24]:
filename='UNSWFireVegResponse_UplandBasalt_AlexThomsen+DK.xlsx'
#wbindex[filename][worksheet][0]

In [25]:
col_def={'visit_id':1, 'visit_date':3, 'scorch height':(15,13,14),
         'tree foliage biomass consumed':(17,), 
         'shrub foliage biomass consumed':(19,), 
         'ground foliage biomass consumed':(21,),
        'tree foliage scorch':(16,), 
         'shrub foliage scorch':(18,), 
         'herb foliage scorch':(20,)}

In [26]:
records=read_fire_intensity(inputdir,filename,worksheet,col_def)

In [27]:
records[0:9]

[{'visit_id': 'CRC09B7U',
  'visit_date': datetime.date(2021, 2, 3),
  'measured_var': 'scorch height',
  'units': 'm',
  'best': 5,
  'lower': 2,
  'upper': 10},
 {'visit_id': 'CRC09B7U',
  'visit_date': datetime.date(2021, 2, 3),
  'measured_var': 'tree foliage biomass consumed',
  'units': '%',
  'best': 10},
 {'visit_id': 'CRC09B7U',
  'visit_date': datetime.date(2021, 2, 3),
  'measured_var': 'shrub foliage biomass consumed',
  'units': '%'},
 {'visit_id': 'CRC09B7U',
  'visit_date': datetime.date(2021, 2, 3),
  'measured_var': 'ground foliage biomass consumed',
  'units': '%',
  'best': 100},
 {'visit_id': 'CRC09B7U',
  'visit_date': datetime.date(2021, 2, 3),
  'measured_var': 'tree foliage scorch',
  'units': '%',
  'best': 5},
 {'visit_id': 'CRC09B7U',
  'visit_date': datetime.date(2021, 2, 3),
  'measured_var': 'shrub foliage scorch',
  'units': '%'},
 {'visit_id': 'CRC09B7U',
  'visit_date': datetime.date(2021, 2, 3),
  'measured_var': 'herb foliage scorch',
  'units': '%',


In [28]:
batch_upsert(params,"form.field_visit_vegetation_estimates",records,
             keycol=('visit_id','visit_date','measured_var'), 
             idx='field_visit_vegetation_estimates_pkey',execute=True)



Connecting to the PostgreSQL database...
252 rows updated
{}
Database connection closed.


This one also has twig diameter measurements:

In [29]:
col_def={'visit_id':1, 'visit_date':3, 'twig diameter':range(23,33)}

In [30]:
records=read_twig_diameters(inputdir,filename,worksheet,col_def)

In [31]:
len(records)

109

In [32]:
records[0]

{'visit_id': 'CRC09B7U',
 'visit_date': datetime.date(2021, 2, 3),
 'measured_variable': 'twig diameter',
 'units': 'mm',
 'single_value': 0.45}

In [33]:
batch_upsert(params,"form.field_visit_vegetation_raw_values",records,
             keycol=('visit_id','visit_date','measured_variable'), 
             idx=None,execute=True)


Connecting to the PostgreSQL database...
109 rows updated
{}
Database connection closed.


#### NE NSW / SE Qld Rainforest
Added a column for date of sampling and changed date format

In [87]:
filename=valid_files[2]
print(filename)
worksheet='Fire'
wbindex[filename][worksheet][0]

UNSW_VegFireResponse_RMK_reformat_Sep2021a.xlsx


['Site',
 'Replicate',
 'Date of samping',
 'Date of last fire dd/mm/yyyy',
 'Date of penultimate fire',
 'Date of earlier fire',
 'How date inferred1',
 'How date inferred2',
 'How date inferred3',
 'Ignition cause1',
 'Ignition cause2',
 'Ignition cause3',
 'Scorch hgt (m)',
 '% Tree foliage scorch',
 "% Tree foliage c'sume",
 '% Shb foliage scorch',
 "% Shb foliage c'sume",
 '% Herb layer foliage scorch',
 "% Herb layer foliage c'sume",
 'Twig diam (mm) 1',
 'Twig diam (mm) 2',
 'Twig diam (mm) 3',
 'Twig diam (mm) 4',
 'Twig diam (mm) 5',
 'Twig diam (mm) 6',
 'Twig diam (mm) 7',
 'Twig diam (mm) 8',
 'Twig diam (mm) 9',
 'Twig diam (mm) 10',
 'Peat depth burnt (cm)',
 'Peat extent burnt %quad']

In [88]:
col_def={'visit_id':1, 'visit_date':3, 'scorch height':(13,),
         'tree foliage biomass consumed':(15,), 
         'shrub foliage biomass consumed':(17,), 
         'ground foliage biomass consumed':(19,),
        'tree foliage scorch':(14,), 
         'shrub foliage scorch':(16,), 
         'herb foliage scorch':(18,),
        'peat extent burnt':(31,),
        'peat depth burnt':(30,)}

In [89]:
records=read_fire_intensity(inputdir,filename,worksheet,col_def)

In [90]:
len(records)
#records[0:2]

153

In [91]:
batch_upsert(params,"form.field_visit_vegetation_estimates",records,
             keycol=('visit_id','visit_date','measured_var'), 
             idx='field_visit_vegetation_estimates_pkey',execute=True)

Connecting to the PostgreSQL database...
153 rows updated
{}
Database connection closed.


In [39]:
col_def={'visit_id':1, 'visit_date':3, 'twig diameter':range(20,30)}
records=read_twig_diameters(inputdir,filename,worksheet,col_def)

In [40]:
len(records)

160

In [41]:
batch_upsert(params,"form.field_visit_vegetation_raw_values",records,
             keycol=('visit_id','visit_date','measured_variable'), 
             idx=None,execute=True)

Connecting to the PostgreSQL database...
160 rows updated
{}
Database connection closed.


#### Yatteyattah
Add a column for the visit date and changed the date format

In [61]:
filename='UNSW_VegFireResponse_DataEntry_Yatteyattah all +DK +Milton_revisedfields_Mar2022.xlsx'
wbindex[filename][worksheet][0]

['Site',
 'Date of last fire dd/mm/yyyy',
 'Date of penultimate fire',
 'Date of earlier fire',
 'How date inferred1',
 'How date inferred2',
 'How date inferred3',
 'Ignition cause1',
 'Ignition cause2',
 'Ignition cause3',
 'Scorch hgt (m)',
 '% Tree foliage scorch',
 "% Tree foliage c'sume",
 '% Shb foliage scorch',
 "% Shb foliage c'sume",
 '% Herb layer foliage scorch',
 "% Herb layer foliage c'sume",
 'Twig diam (mm) 1',
 'Twig diam (mm) 2',
 'Twig diam (mm) 3',
 'Twig diam (mm) 4',
 'Twig diam (mm) 5',
 'Twig diam (mm) 6',
 'Twig diam (mm) 7',
 'Twig diam (mm) 8',
 'Twig diam (mm) 9',
 'Twig diam (mm) 10',
 'Peat depth burnt (cm)',
 'Peat extent burnt %quad']

In [62]:
col_def={'visit_id':1, 'visit_date':2, 'scorch height':(12,),
         'tree foliage biomass consumed':(14,), 
         'shrub foliage biomass consumed':(16,), 
         'ground foliage biomass consumed':(18,),
        'tree foliage scorch':(13,), 
         'shrub foliage scorch':(15,), 
         'herb foliage scorch':(17,)}

In [63]:
col_def

{'visit_id': 1,
 'visit_date': 2,
 'scorch height': (12,),
 'tree foliage biomass consumed': (14,),
 'shrub foliage biomass consumed': (16,),
 'ground foliage biomass consumed': (18,),
 'tree foliage scorch': (13,),
 'shrub foliage scorch': (15,),
 'herb foliage scorch': (17,)}

In [66]:
records=read_fire_intensity(inputdir,filename,worksheet,col_def)

In [67]:
len(records)
records[0:2]

[{'visit_id': 'SCCJB37-Near',
  'visit_date': datetime.date(2020, 7, 12),
  'measured_var': 'scorch height',
  'units': 'm',
  'best': 3},
 {'visit_id': 'SCCJB37-Near',
  'visit_date': datetime.date(2020, 7, 12),
  'measured_var': 'tree foliage biomass consumed',
  'units': '%'}]

In [68]:
batch_upsert(params,"form.field_visit_vegetation_estimates",records,
             keycol=('visit_id','visit_date','measured_var'), 
             idx='field_visit_vegetation_estimates_pkey',execute=True)

Connecting to the PostgreSQL database...
63 rows updated
{}
Database connection closed.


In [69]:
col_def={'visit_id':1, 'visit_date':2, 'twig diameter':range(19,29)}
records=read_twig_diameters(inputdir,filename,worksheet,col_def)

In [70]:
len(records)

70

In [71]:
batch_upsert(params,"form.field_visit_vegetation_raw_values",records,
             keycol=('visit_id','visit_date','measured_variable'), 
             idx=None,execute=True)

Connecting to the PostgreSQL database...
70 rows updated
{}
Database connection closed.


#### Alpine Ash
Information about fire history and fire intensity are incomplete in this file

In [72]:
valid_files[4]

'UNSW_VegFireResponse_KNP AlpAsh_firehistupdate.xlsx'

#### Alpine Bogs

In [77]:
filename=valid_files[5]
print(filename)
worksheet='Fire'

UNSW_VegFireResponse_AlpineBogs_reformat_Sep2021.xlsx


In [78]:
col_def={'visit_id':1, 'visit_date':3, 
         'shrub foliage biomass consumed':(17,), 
         'ground foliage biomass consumed':(19,),
        'peat extent burnt':(31,),
        'peat depth burnt':(30,)}

In [81]:
records=read_fire_intensity(inputdir,filename,worksheet,col_def)

In [82]:
len(records)

24

In [83]:
batch_upsert(params,"form.field_visit_vegetation_estimates",records,
             keycol=('visit_id','visit_date','measured_var'), 
             idx='field_visit_vegetation_estimates_pkey',execute=True)


Connecting to the PostgreSQL database...
24 rows updated
{}
Database connection closed.


In [84]:
col_def={'visit_id':1, 'visit_date':3, 'twig diameter':range(20,30)}
records=read_twig_diameters(inputdir,filename,worksheet,col_def)

In [86]:
len(records)
batch_upsert(params,"form.field_visit_vegetation_raw_values",records,
             keycol=('visit_id','visit_date','measured_variable'), 
             idx=None,execute=True)

Connecting to the PostgreSQL database...
60 rows updated
{}
Database connection closed.


#### Robertson RF

In [93]:
filename=valid_files[6]
print(filename)


RobertsonRF_data_bionet2.xlsx


In [94]:
col_def={'visit_id':1, 'visit_date':3, 'scorch height':(15,13,14),
         'tree foliage biomass consumed':(17,), 
         'shrub foliage biomass consumed':(19,), 
         'ground foliage biomass consumed':(21,),
        'tree foliage scorch':(16,), 
         'shrub foliage scorch':(18,), 
         'herb foliage scorch':(20,),
         'peat extent burnt':(33,),
        'peat depth burnt':(32,)}

In [97]:
records=read_fire_intensity(inputdir,filename,worksheet,col_def)

In [98]:
batch_upsert(params,"form.field_visit_vegetation_estimates",records,
             keycol=('visit_id','visit_date','measured_var'), 
             idx='field_visit_vegetation_estimates_pkey',execute=True)


Connecting to the PostgreSQL database...
18 rows updated
{}
Database connection closed.


In [99]:
col_def={'visit_id':1, 'visit_date':3, 'twig diameter':range(22,32)}
records=read_twig_diameters(inputdir,filename,worksheet,col_def)

In [101]:
len(records)
batch_upsert(params,"form.field_visit_vegetation_raw_values",records,
             keycol=('visit_id','visit_date','measured_variable'), 
             idx=None,execute=True)

Connecting to the PostgreSQL database...
20 rows updated
{}
Database connection closed.
